# Problema 1

In [54]:
import os
from PIL import Image
import numpy as np
import cv2

def invert_colors(img):
    # Invertir los colores de la imagen
    return 255 - img

def bradley_roth_adaptive_threshold(image, s=25, t=0.08):
    # Convertir imagen a escala de grises si es necesario
    if image.mode != 'L':
        image = image.convert('L')
    img = np.array(image, dtype=np.float64)

    # Calcular las sumas integrales
    integral_img = np.cumsum(np.cumsum(img, axis=1), axis=0)

    # Definir tamaño de la ventana
    rows, cols = img.shape
    s2 = s // 2
    result = np.zeros_like(img)

    for i in range(rows):
        for j in range(cols):
            # Definir límites de la ventana
            x1 = max(j - s2, 0)
            x2 = min(j + s2, cols - 1)
            y1 = max(i - s2, 0)
            y2 = min(i + s2, rows - 1)

            count = (y2 - y1) * (x2 - x1)
            sum_ = integral_img[y2, x2] - integral_img[y1, x2] - integral_img[y2, x1] + integral_img[y1, x1]

            if img[i, j] * count <= sum_ * (1 - t):
                result[i, j] = 0
            else:
                result[i, j] = 255

    # Invertir colores justo antes de retornar la imagen
    result = invert_colors(result)
    return Image.fromarray(result.astype(np.uint8))

def process_images_in_folder(folder_path):
    for i in range(1, 21):  # Suponiendo 20 imágenes
        original_path = os.path.join(folder_path, f'{i}.pgm')
        
        if os.path.exists(original_path):
            image = Image.open(original_path)
            binarized_image = bradley_roth_adaptive_threshold(image)
            binarized_image.save(os.path.join(folder_path, f'{i}_bn.pgm'))
            print(f'Imagen procesada y guardada: {i}_bn.pgm')
        else:
            print(f'Imagen no encontrada: {original_path}')

# Actualizar con la ruta de tu carpeta
folder_path = './database'
process_images_in_folder(folder_path)


Imagen procesada y guardada: 1_bn.pgm
Imagen procesada y guardada: 2_bn.pgm
Imagen procesada y guardada: 3_bn.pgm
Imagen procesada y guardada: 4_bn.pgm
Imagen procesada y guardada: 5_bn.pgm
Imagen procesada y guardada: 6_bn.pgm
Imagen procesada y guardada: 7_bn.pgm
Imagen procesada y guardada: 8_bn.pgm
Imagen procesada y guardada: 9_bn.pgm
Imagen procesada y guardada: 10_bn.pgm
Imagen procesada y guardada: 11_bn.pgm
Imagen procesada y guardada: 12_bn.pgm
Imagen procesada y guardada: 13_bn.pgm
Imagen procesada y guardada: 14_bn.pgm
Imagen procesada y guardada: 15_bn.pgm
Imagen procesada y guardada: 16_bn.pgm
Imagen procesada y guardada: 17_bn.pgm
Imagen procesada y guardada: 18_bn.pgm
Imagen procesada y guardada: 19_bn.pgm
Imagen procesada y guardada: 20_bn.pgm


In [56]:
import numpy as np
import cv2
import os

def calculate_metrics(gt, pred):
    TP = np.sum((gt == 255) & (pred == 255))
    TN = np.sum((gt == 0) & (pred == 0))
    FP = np.sum((gt == 0) & (pred == 255))
    FN = np.sum((gt == 255) & (pred == 0))

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    f1_score = 2*TP / (2*TP + FP + FN) if (2*TP + FP + FN) != 0 else 0

    return accuracy, sensitivity, specificity, precision, f1_score

folder_path = './database'

metrics = []

for i in range(1, 21):
    gt_path = os.path.join(folder_path, f'{i}_gt.pgm')
    bn_path = os.path.join(folder_path, f'{i}_bn.pgm')
    
    if os.path.exists(gt_path) and os.path.exists(bn_path):
        gt = cv2.imread(gt_path, 0)
        pred = cv2.imread(bn_path, 0)
        
        metrics.append(calculate_metrics(gt, pred))
    else:
        metrics.append((None, None, None, None, None))

print("{:<6} | {:<9} | {:<11} | {:<11} | {:<9} | {:<7}".format("Image", "Accuracy", "Sensitivity", "Specificity", "Precision", "F1 Score"))
print("-" * 70)  # Imprime una línea separadora

for i, (accuracy, sensitivity, specificity, precision, f1_score) in enumerate(metrics, 1):
    print("{:<6} | {:<9.3f} | {:<11.3f} | {:<11.3f} | {:<9.3f} | {:<7.3f}".format(i, accuracy or 0, sensitivity or 0, specificity or 0, precision or 0, f1_score or 0))


Image  | Accuracy  | Sensitivity | Specificity | Precision | F1 Score
----------------------------------------------------------------------
1      | 0.946     | 0.735       | 0.956       | 0.421     | 0.536  
2      | 0.943     | 0.702       | 0.954       | 0.433     | 0.535  
3      | 0.952     | 0.703       | 0.963       | 0.455     | 0.552  
4      | 0.965     | 0.352       | 0.993       | 0.705     | 0.470  
5      | 0.934     | 0.793       | 0.940       | 0.331     | 0.467  
6      | 0.930     | 0.762       | 0.937       | 0.314     | 0.444  
7      | 0.949     | 0.834       | 0.955       | 0.490     | 0.617  
8      | 0.949     | 0.736       | 0.960       | 0.472     | 0.575  
9      | 0.931     | 0.832       | 0.936       | 0.411     | 0.550  
10     | 0.869     | 0.565       | 0.883       | 0.181     | 0.274  
11     | 0.915     | 0.616       | 0.928       | 0.265     | 0.371  
12     | 0.944     | 0.739       | 0.962       | 0.626     | 0.678  
13     | 0.945     | 0.866     